In [ ]:
"""
安裝及使用教學： https://github.com/nltk/nltk/wiki/Stanford-CoreNLP-API-in-NLTK

安裝CoreNLP，總共要下載3個檔案
1. stanford-corenlp-full-2018-10-05
2. stanford-chinese-corenlp-2018-10-05-models
3. StanfordCoreNLP-chinese.properties

前兩個檔案下載網址 https://stanfordnlp.github.io/CoreNLP/
第三個檔案下載網址 https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/src/edu/stanford/nlp/pipeline/StanfordCoreNLP-chinese.properties

注意：要先把第一個檔案解壓縮之後，再把另外兩個檔案放到同一個資料夾
"""

"""
在進入python使用CoreNLP套件之前
1. change path to stanford-corenlp-full-2018-10-05 directory
2. start server: 
    java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer \
    -preload tokenize,ssplit,pos,lemma,ner,parse,depparse \
    -status_port 9000 -port 9000 -timeout 1500000 & 
"""

In [4]:
import os
import re 
import jieba
import pandas as pd
from opencc import OpenCC
from nltk.parse import CoreNLPParser

In [2]:
parser = CoreNLPParser(url = 'http://localhost:9001', tagtype = 'ner')   # tagtype = 'pos' or 'ner'

In [3]:
content = '高雄醫學大學去年7月校務會議通過對第16屆董事會在8、9年前，數度修改捐助章程，不實登載「陳啟川為創辦人」，經提起偽造文書訴訟，5日首度在高雄地檢署開庭。10多位校友趕來聲援，拉布條、高喊「解散董事會」。高醫大董事會5日則聲明，無任何偽造文書事實，已故陳啟川先生捐出10餘甲土地，讓高醫大順利設校史實不容抹煞；高醫現有土地所有權狀大部分登載轉移原因為「捐贈」，是由陳啟川母親擔任董事長的南和興產公司捐贈。高醫大校友藍傳盛表示，台灣省教育廳1954年函轉教育部案由「私立高雄醫學院創辦人杜聰明呈送設院事項」，附件即載明杜聰明詳細履歷及擬聘董事名單，因此學校創辦人為杜聰明毫無疑問，校友會立場支持學校提告。'
keyword = '偽造文書'
sentences = []
for key in re.finditer(keyword, content):
    end_list = ['。', '！', '!', '？', '?']
    sentence_start = 0
    sentence_end = len(content) - 1
    for end in end_list:
        dis_to_keystart = content[:key.start()][::-1].find(end)
        if dis_to_keystart != -1: sentence_start = max(sentence_start, key.start() - dis_to_keystart) 
        dis_to_keyend = content[key.end():].find(end)
        if dis_to_keyend != -1: sentence_end = min(sentence_end, key.end() + dis_to_keyend)
    
    sentences.append(content[sentence_start: sentence_end+1])

print(sentences)

['高雄醫學大學去年7月校務會議通過對第16屆董事會在8、9年前，數度修改捐助章程，不實登載「陳啟川為創辦人」，經提起偽造文書訴訟，5日首度在高雄地檢署開庭。', '高醫大董事會5日則聲明，無任何偽造文書事實，已故陳啟川先生捐出10餘甲土地，讓高醫大順利設校史實不容抹煞；高醫現有土地所有權狀大部分登載轉移原因為「捐贈」，是由陳啟川母親擔任董事長的南和興產公司捐贈。']


In [4]:
# (Optional) convert from Simplified Chinese to Traditional Chinese
cc = OpenCC('t2s') 
for sentence in sentences:
    sentence = cc.convert(sentence)
    nltk_cut = list(parser.tokenize(sentence))
    jieba_cut = list(jieba.cut(sentence, cut_all = False))
    print('nltk_cut:', nltk_cut, '\n')
    print('jieba_cut:', jieba_cut, '\n')
    
    nltk_tag = list(parser.tag(nltk_cut))
    print('nltk_tag:', nltk_tag, '\n')
    
    per = []
    org = []
    for wordtag_pair in nltk_tag:
        index = nltk_tag.index(wordtag_pair)
        
        word = wordtag_pair[0]
        tag = wordtag_pair[1]
        if tag == 'PERSON':
            if nltk_tag[index-1][1] == 'PERSON': per[-1] += word
            else: per.append(word)
            
        if tag == 'ORGANIZATION':
            if nltk_tag[index-1][1] == 'ORGANIZATION': org[-1] += word
            else: org.append(word)
        
    print(org, '\n')
    print(per, '\n')
    
    # nltk_sentence = list(parser.parse(nltk_cut))
    # print('nltk_tag:', nltk_sentence, '\n''\n')

ConnectionError: HTTPConnectionPool(host='localhost', port=9001): Max retries exceeded with url: /?properties=%7B%22outputFormat%22%3A+%22json%22%2C+%22annotators%22%3A+%22tokenize%2Cssplit%22%7D (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000226906BCA58>: Failed to establish a new connection: [WinError 10061] 無法連線，因為目標電腦拒絕連線。',))

In [5]:
nltk_sentence = list(parser.parse(nltk_cut))
print('nltk_sentence:', nltk_sentence, '\n''\n')

NameError: name 'nltk_cut' is not defined

In [ ]:
for line in nltk_sentence:
    for sentence in line:
        sentence.draw()

In [15]:
from nltk.parse import CoreNLPParser
import jieba
from opencc import OpenCC
parser = CoreNLPParser(url='http://localhost:9000', tagtype = 'pos')
to_convert="台北市政府工務局衛生下水道工程處前科長吳錦堂，利用下水道整治、瀝青舖設工程索賄，過程中，吳嫌賄款少，竟嗆廠商「我是8職等的科長」、「隨時抽料讓材料不過關」，共索賄42萬元，高等法院今依貪污治罪條例重判吳15年6月重罪、褫奪公權4年"
cc = OpenCC('t2s')  # (Optional )convert from Simplified Chinese to Traditional Chinese
content = cc.convert(to_convert)#一樣轉成簡體字，這樣Stanford的系統才會比較準確

word_list = list(jieba.cut(content, cut_all=False))
tree_list = parser.parse(word_list)


for trees in tree_list:
    for tree in trees:
        
        print('取得tree的label:')
        print(tree.treepositions())
        print("-"*120)
        
        print('畫出tree來:')
        tree.pretty_print()
        print("-"*120)
        
        print('用文字的方式來表達tree:')
        parse_string = ' '.join(str(tree).split()) 
        print(parse_string)
        print("-"*120)
        
        print('用內建的畫圖方式畫出tree，會跳出新視窗')
        tree.draw()

取得tree的label:
[(), (0,), (0, 0), (0, 0, 0), (0, 0, 0, 0), (0, 0, 0, 0, 0), (0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 0, 0), (0, 0, 1, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1), (0, 0, 1, 0, 0, 0, 0, 1, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1), (0, 0, 1, 0, 0, 0, 0, 1, 0